In [62]:
import torch
import torch.nn.functional as F
from Bio import SeqIO
import numpy as np
from tqdm import tqdm, trange

from pathlib import Path
from collections import Counter
from itertools import product

from lightning_module import MetaLightningModule

In [63]:
CHECKPOINT_ROOT = '/charonfs/scratch/users/astar/gis/stanojevicd/data/meta/training/meta_classifier/ozfuu58l/checkpoints'
MODEL_FILE  = 'step=50000-val_loss=0.299029.ckpt'
DEVICE = torch.device('cuda:0')

model = MetaLightningModule.load_from_checkpoint(Path(CHECKPOINT_ROOT, MODEL_FILE), map_location=DEVICE).eval().to(dtype=torch.bfloat16)

/charonfs/home/users/astar/gis/stanojevicd/mambaforge/envs/meta/lib/python3.12/site-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


In [67]:
DATA_ROOT = '/charonfs/scratch/users/astar/gis/stanojevicd/data/meta/databases/MetageNN_main/main_database'
files = sorted(list(Path(DATA_ROOT).glob('*.fasta')))

len(files)

516

In [64]:
BASES_ENCODING = {b: i for i, b in enumerate('ACGT')}
KMER_ENCODING = {''.join(k): i for i, k in enumerate(product('ACGTN', repeat=5))}

files = ['/charonfs/scratch/users/astar/gis/stanojevicd/data/meta/datasets/MetageNN_main/SRA_IN/SRR11070451.fasta']

all_results = []
for file in tqdm(files[:100]):
    for record in SeqIO.parse(file, 'fasta'):
        array = ''.join([c if c in 'ACGT' else 'N' for c in str(record.seq).upper()])
        results = []

        with torch.no_grad():
            batch = []
            for start in range(0, len(array) - 1000 + 1, 1000):
                s = array[start:start+1000]
                s = torch.tensor(
                    [
                        KMER_ENCODING[s[i : i + 5]] if i != -5 else len(KMER_ENCODING)
                        for i in range(-5, 1000, 5)
                    ]
                )
                batch.append(s)

                if len(batch) == 2048:
                    x = torch.stack(batch, dim=0).to(device=DEVICE)
                    y = model(x).softmax(dim=1)

                    results.append(y.to(device='cpu', dtype=torch.float16).numpy())
                    batch = []
            
            if len(batch) > 0:
                x = torch.stack(batch, dim=0).to(device=DEVICE)
                y = model(x).softmax(dim=1)

                results.append(y.to(device='cpu', dtype=torch.float16).numpy())
        
        all_results.append(results)

  0%|                                                                                                                                                                                           | 0/1 [02:56<?, ?it/s]


KeyboardInterrupt: 

In [65]:
for results in all_results:
    counter, total, hc = Counter(), 0, 0
    for result in results:
        # e_idx, seq_idx = np.nonzero(result >= 0.5)
        #counter.update(seq_idx.tolist())

        argmax = result.argmax(axis=1)
        confident_idx = np.nonzero(result[range(len(result)), argmax] > 0.5)[0]

        counter.update(argmax[confident_idx].tolist())
        total += len(result)
        # hc += len(seq_idx)

    print(counter.most_common(5), total)

[] 0
[] 0
[] 0
[(345, 10), (260, 1), (183, 1), (252, 1)] 24
[(345, 1)] 1
[(345, 3), (226, 2), (401, 1)] 7
[(345, 4), (354, 1), (58, 1)] 7
[] 0
[(345, 5), (476, 1), (252, 1), (219, 1), (208, 1)] 17
[] 0
[(345, 1)] 1
[] 0
[(345, 3), (127, 1)] 7
[(345, 1)] 2
[(109, 1)] 1
[(486, 1)] 4
[(30, 2), (345, 2)] 4
[] 0
[] 0
[(345, 16), (162, 1), (109, 1), (272, 1)] 22
[(345, 1)] 1
[] 0
[(345, 35), (208, 2), (98, 2), (513, 1), (365, 1)] 55
[(345, 2), (401, 1)] 3
[(345, 4)] 5
[(345, 3)] 3
[] 1
[] 0
[(345, 10), (98, 1), (350, 1)] 13
[] 0
[(345, 6), (174, 1)] 11
[] 0
[(345, 6), (513, 1)] 8
[] 0
[] 0
[(345, 2)] 3
[(345, 1)] 3
[(208, 1)] 2
[(345, 1)] 1
[(345, 1)] 1
[] 0
[(345, 6), (396, 1), (376, 1)] 12
[] 0
[(345, 1)] 1
[(109, 1), (345, 1)] 2
[(345, 2), (30, 1)] 3
[(345, 2), (396, 1), (30, 1)] 4
[(345, 5)] 6
[] 0
[(220, 1), (345, 1)] 3
[(345, 18), (75, 2), (208, 2), (476, 1), (305, 1)] 35
[] 0
[(345, 8), (296, 1)] 9
[(345, 1)] 2
[(345, 3), (82, 1), (139, 1), (279, 1)] 8
[(345, 1)] 3
[(208, 1)] 3
[] 0
[

In [37]:
np.max(all_results[2][0], axis=1).tolist()

[0.98046875,
 0.73046875,
 0.62109375,
 0.5625,
 0.6796875,
 0.62109375,
 0.77734375,
 0.62109375,
 0.84765625,
 0.5625,
 0.62109375,
 0.77734375,
 0.5625,
 0.5625,
 0.77734375,
 0.62109375,
 0.73046875,
 0.62109375,
 0.5625,
 0.62109375,
 0.62109375,
 0.77734375,
 0.5,
 0.73046875,
 0.77734375,
 0.5,
 0.5,
 0.5625,
 0.75390625,
 0.5,
 0.5625,
 0.73046875,
 0.5625,
 0.62109375,
 0.6796875,
 0.77734375,
 0.6796875,
 0.6796875,
 0.62109375,
 0.6796875,
 0.81640625,
 0.6796875,
 0.73046875,
 0.73046875,
 0.62109375,
 0.6796875,
 0.73046875,
 0.8515625,
 0.5625,
 0.77734375,
 0.5625,
 0.62109375,
 0.62109375,
 0.77734375,
 0.73046875,
 0.81640625,
 0.90625,
 0.87890625,
 0.77734375,
 0.5,
 0.8515625,
 0.62109375,
 0.6796875,
 0.77734375,
 0.6796875,
 0.62109375,
 0.62109375,
 0.5625,
 0.5625,
 0.62109375,
 0.81640625,
 0.96875,
 0.5,
 0.67578125,
 0.8515625,
 0.62109375,
 0.96875,
 0.6796875,
 0.5546875,
 0.77734375,
 0.65234375,
 0.81640625,
 0.6796875,
 0.90625,
 0.5625,
 0.5625,
 0.7304

In [73]:
files[22]

PosixPath('/charonfs/scratch/users/astar/gis/stanojevicd/data/meta/databases/MetageNN_main/main_database/GCF_000020105.1_ASM2010v1.fasta')

In [17]:
all_lens = []
for file in tqdm(files[:100]):
    record = max(SeqIO.parse(file, 'fasta'), key=lambda r: len(r))
    all_lens.append(len(record))

sorted(range(100), key=lambda i: all_lens[i], reverse=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 73.99it/s]


[89,
 94,
 9,
 74,
 83,
 8,
 31,
 4,
 44,
 95,
 5,
 20,
 88,
 70,
 23,
 25,
 46,
 96,
 79,
 18,
 29,
 15,
 92,
 58,
 14,
 7,
 47,
 13,
 68,
 34,
 40,
 98,
 28,
 0,
 41,
 73,
 66,
 2,
 75,
 62,
 16,
 65,
 38,
 36,
 61,
 37,
 11,
 71,
 6,
 21,
 54,
 85,
 33,
 57,
 12,
 99,
 45,
 39,
 97,
 19,
 1,
 82,
 87,
 60,
 27,
 53,
 76,
 72,
 50,
 90,
 10,
 59,
 64,
 52,
 55,
 22,
 48,
 42,
 24,
 69,
 56,
 49,
 35,
 43,
 17,
 3,
 67,
 30,
 81,
 80,
 77,
 51,
 32,
 63,
 91,
 78,
 86,
 26,
 84,
 93]